In [ ]:
import requests
import json
import os

# mettre son appID et sa key puis supprimer.
appid = "a5b42571"
key = "1dc250b1f29e8bd11f6f3c5cb995bf71"
url = "http://api.adzuna.com/v1/api/jobs/fr/categories?app_id={0}&app_key={1}&&content-type=application/json".format(appid, key)
req1 = requests.get(url).json()

categories = req1["results"]


tags = [] # j'initie cette liste qui récupèrera les tags qui correspondent aux catégories

for cat in categories:
    tags.append(cat["tag"]) # Pour chaque catégorie je récupère son tag

print(tags)